
<p> <center> <a href="../Start_Here.ipynb">Home Page</a> </center> </p>

 
<div>
    <span style="float: left; width: 51%; text-align: right;">
        <a >1</a>
        <a href="2.1.System-Topology.ipynb">2</a>
        <a href="3.Hands-on-Multi-GPU.ipynb">3</a>
        <a href="4.Convergence.ipynb">4</a>
    </span>
    <span style="float: left; width: 49%; text-align: right;"><a href="2.1.System-Topology.ipynb">Next Notebook</a></span>
</div>



# Introduction to Distributed Deep Learning

**Contents of this notebook:**

- [The need for Distributed Deep Learning](#The-need-for-Distributed-Deep-Learning)
- [Differnet types of Distributed Deep learning and it's applications](#Differnet-types-of-Distributed-Deep-learning-and-it's-applications)
    - [Training and Inference](#Training-and-Inference)
    - [Data and Model Parallelism](#Data-and-Model-Parallelism)
    - [Framework and NVIDIA NGC Support - Optional](#Framework-and-NVIDIA-NGC-Support---Optional)
- [Demo - Scalability across multiple GPUs](#Demo---Scalability-across-multiple-GPUs) 


**The objectives of this Notebook is to make understand:**

- The need for distributed Deep Learning.
- The ecosystem of distributed Deep Learning.

# The need for Distributed Deep Learning

Artificial Intelligence has witnessed tremendous progress in the past decade, making its impact in almost every potential field. Still, the concepts for the backbone of Neural networks such as Perceptron, backpropagation were published in 1958 and 1974 respectively. They were not widespread because of the unavailability of computing power and lack of data transfer and storage systems. The introduction of the Internet and the recent improvements in compute have both been a key component of AI progress in recent times, with [NVIDIA](https://www.nvidia.com/) being the leader in AI progress in the modern world.

With the recent advancements in computing power, modern Deep neural networks are capable of processing a wide variety of data. They can do a wide range of tasks, but these Deep learning workloads have also significantly grown in size. This makes deep learning applications time-intensive for larger workloads.

The below chart plots the amount of computing required by various modern networks starting from AlexNet to AlphaGo Zero.

<center><img src="images/ai-and-compute-modern-log.png"/></center>


This chart points out that AI training runs has been increasing exponentially with a 3.4-month doubling time, by comparison to [Moore's Law](https://en.wikipedia.org/wiki/Moore%27s_law) which has a doubling period of 2-years, this difference might make it practically hard to train larger networks, so to enable researchers and data scientists to train bigger networks with higher compute power, [NVIDIA](https://www.nvidia.com/) constantly innovates in both software and hardware forefronts bringing out new technology such as [AMP](https://developer.nvidia.com/automatic-mixed-precision) and [NVIDIA Tensor Cores](https://www.nvidia.com/en-in/data-center/tensor-cores/).

- [AMP](https://developer.nvidia.com/automatic-mixed-precision): Deep Neural Network training has traditionally relied on IEEE single-precision format. However, you can train with mixed precision with half-precision while maintaining the network accuracy achieved with single precision. This technique of using both `single` and `half-precision` representations is referred to as the mixed-precision technique. The benefits are : 
    - Speeds up math-intensive operations, such as linear and convolution layers, using [NVIDIA Tensor Cores](https://www.nvidia.com/en-in/data-center/tensor-cores/).
    - Speeds up memory-limited operations by accessing half the bytes compared to single-precision.
    - Reduces memory requirements for training models, enabling larger models or larger mini-batches.
    
- [NVIDIA Tensor Cores](https://www.nvidia.com/en-in/data-center/tensor-cores/): Tensor Cores enable mixed-precision computing, dynamically adapting calculations to accelerate throughput while preserving accuracy. The latest generation expands these speedups to a full range of workloads. From 10X speedups in AI training with Tensor Float 32 (TF32), a revolutionary new precision, to 2.5X boosts for high-performance computing with floating-point 64 (FP64)

GPUs are the choice of AI researchers and data scientists for their ability to perform massive parallelism and high throughput. 

Before going further, let us define what we mean by **throughput**. Throughput refers to the number of data units processed per unit of time. The data unit varies according to our application. For example, if we have a Computer Vision application, we calculate the images/sec processed through the deep learning network. By having a higher throughput, we process more data units, which leads us to faster convergence of our system.


Let us now do an experiment and calculate the throughput of training a Natural language processing with different batch sizes in a single GPU.

In [1]:
SINGULARITY_RUN="singularity run --nv --env TF_CPP_MIN_LOG_LEVEL=3 ~/DDL.simg "

In [2]:
# GPT - Wikitext -2 : 32 ,64 , 128 , 256 ,512 , 1024
COMMAND = SINGULARITY_RUN + ' python3 ../source_code/N1/GPT.py --batch-size=32 2> /dev/null' 
!echo  $COMMAND > command && srun --partition=gpu -n1 --gres=gpu:1 /bin/bash ./command

srun: job 3579 queued and waiting for resources
srun: job 3579 has been allocated resources
Batch size: 32
Epoch 1/3
1402/1402 [==============================] - 50s 33ms/step - loss: 2.2799 - dense_2_loss: 2.2799..1721 - dense_2_loss: 3.1 loss: 2.9933 - dense_2_loss: 2.- dense_2_loss: 2.8known - 12s 32ms/step - loss: 2.7492 - dense_2_loss: 2.749: 2.7ms/step - loss: 2.6575 - dense_2_loss: 2.: 2.6199 - dense_2_loss: 2.61   356/Unknown - 16s 33ms/step - loss: 2.5680 - dense_2_loss: 2.568se_2_loss: 2.54 - 17s 33ms/step - loss: 2.5249 - dense_2_loss: 2.5240ep - loss: 2.4845 - dense_2_loss: 2.808 - dense_2_loss: 2.47/Unknown - 21s 33ms/step - loss: 2.4735 - dense_2_loss: 2.473loss: 2.4s 33ms/step - loss: 2.4461 - dense_2_loss: 2.loss: 2.4333 - dense_2_loss: 2. dense_2_loss: 2.41nown - 27s 33ms/step - loss: 2.3931 - dense_2_loss: 2.393 2.38s/step - loss: 2.3718 - dense_2_loss: 2. 2.3742 - dense_2_loss: 2.3  797/Unknown - 30s 33ms/step - loss: 2.3663 - dense_2_loss: 2.366e_2_loss: 2.3- 32s 33

Let us now tabulate the results that we obtained in the above experiment. Ignore the first iteration as it includes graph building time and one-time operations in them.

Output of running the command on DGX-1 (V100): 

|Batch Size| Throughput |
|-|-|
|32|1050|
|64|1498|
|128|1906|
|256|2192|
|512|2364|
|1024|2452|
|1280|2456|

Output of running the command on AMPERE (A100): 

|Batch Size| Throughput |
|-|-|
|32|3995|
|64|5303|
|128|6383|
|256|7100|
|512|7794|
|1024|8108|
|1280|8161|

We can noticed that the throughput increases as we initially increase the batch size and reaches a ceiling at which we have completely utilised the compute or memory throughput available to us.

A straightforward method to increase training throughput is to use multiple GPU devices to increase parallelism further; the below chart demonstrates the performance improvements with increase in throughput of images processed by the two different deep neural networks. 

<center><img src="images/horovod_exascale_2.png"/></center>

Now that we understand the need for more computing power for modern networks and how multiple GPUs can bridge this gap with parallelism, let us now understand the types of Distributed Deep learning and their respective applications.

# Differnet types of Distributed Deep learning and it's applications

### Training and Inference

- **Training**: This is the widely used case for distributed deep learning called distributed Training. When the computing power needed for model convergence gets higher, multiples GPUs are then used to increase parallelism and thus reduce the training time. 

- **Inference**: Deep learning inference is the process of using a trained DNN model to make predictions against previously unseen data. Distributed inferencing is used in applications that require low latency and high throughput. One such example is running Inference on multiple video streams on an Intelligent Video Analytics application built with [NVIDIA DeepStream SDK](https://developer.nvidia.com/deepstream-sdk). 


***

### Data and Model Parallelism

- **Model Parallelism**: Model parallelism is the process of splitting a model up between multiple devices or nodes and creating an efficient pipeline to train the model across these devices to maximize GPU utilization. An example representation of model parallelism can be as follows.

<center><img src="images/model_parallelism.jpg"/></center>


- **Data Parallelism**: In modern deep learning, when the dataset is too big to be fit into the memory, we could only do stochastic gradient descent for batches. The shortcoming of stochastic gradient descent is that the estimate of the gradients might not accurately represent the true gradients of using the full dataset. Therefore, it may take much longer to converge. A natural way to have a more accurate estimate of the gradients is to use larger batch sizes or even use the full dataset. To allow this, the gradients of small batches are calculated on each GPU. The final estimate of the gradients is the weighted average of the gradients calculated from all the small batches. 

    - **Synchronous data parallelism** : In synchronous data parallelism, all workers train over different slices of input data in sync and aggregate gradients at each step.
    - **Asynchronous data parallelism** : In synchronous data parallelism, all workers are independently training over the input data and updating variables asynchronously. 
    
Optional: Typically `sync` training is supported via all-reduce and `async` through parameter server architecture


Example representation of Synchronous and Asynchronous data parallelism :

<table><tr>
<td> <img src="images/sync.png" alt="Drawing" /> </td>
<td> <img src="images/async.png" alt="Drawing" /> </td>
</tr></table>


- **Hybrid Parallelism** : Hybrid parallelism is used when we would like to make use of both Data and Model parallelism. An example would be when we need to train a large model that cannot fit into one GPU but can fit into a node, we could use Model parallelism inside a node and using Data parallelism across nodes.

***


### Framework and NVIDIA NGC Support - Optional

Let us look into some frameworks that support Distributed Deep learning. 

#### Frameworks :

- **Tensorflow & Keras** : We can distribute deep learning training using minimal code changes using the `tf.distribute` API to distribute training across multiple GPUs, multiple machines or TPUs. We will look into some strategies that `tf.distribute` API offers in the upcoming notebooks 
- **PyTorch** : PyTorch enables users to distrubute their training using `torch.nn.DataParallel` and `torch.nn.parallel.DistributedDataParallel` for Data parallelism. 
- **MXNet** : MXNet uses `KV Store` server for distributed training. It has 4 modes that determine how the weights are updated and determine where the model is stored.
- **Horovod** : Horovod is a distributed deep learning training framework for TensorFlow, Keras, PyTorch, and Apache MXNet. Horovod was originally developed by Uber to make distributed deep learning fast and easy to use. With Horovod, an existing training script can be scaled up to run on hundreds of GPUs in just a few lines of Python code.

#### NVIDIA NGC Support 

The NVIDIA NGC Catalog is a curated set of GPU-optimized software for AI, HPC and Visualization. The content provided by NVIDIA and third-party ISVs simplify building, customizing, and integrating GPU-optimized software into workflows, accelerating the time to solutions for users. The NGC Catalog consists of containers, pre-trained models, Helm charts for Kubernetes deployments and industry-specific AI toolkits with software development kits (SDKs). NGC Catalog contains containers that have Multi-node support built-in and can be readily deployed. 


We will be primarily looking into using Distributed Deep learning training using Data parallelism in the upcoming notebooks. We will use both Horovod and Tensorflow for this, so the reader can choose whichever framework they would like to follow through. 

# Demo - Scalability across multiple GPUs

Now that we've have seen the basics around the Distributed deep learning system , let us now try running a demo and see how well it scales with multiple GPUs , this is done by calculating a term called as scaling efficiency. 

Scaling efficiency can be defined as follows :

$$
Scaling\ efficiency = \frac{ \frac{Total\ samples\ processed\ per\ unit\ time}{Number\ of\ gpus} }{ \  \ Samples\ processed\ per\ unit\ time\ per\ gpu }
$$

**Note : Scaling efficiency is usually less than 1 because of the additional communication that has to be performed to keep the system in sync.**

Let us now try scaling a simple CNN clasifier with the FMNIST dataset using Synchronous training with Horovod and calcualte it's scaling efficiency run across 1,2,4 and 8 GPUs respectively. You can execute this approach based on your accessibility to either DGX `V100` or `A100`. 



### DGX-1 A100

In [3]:
# 1 GPU 
COMMAND = SINGULARITY_RUN + ' horovodrun -np 1 --mpi-args="--oversubscribe" python3 ../source_code/N1/cnn_fmnist.py --batch-size=8192 2> /dev/null' 
!echo  $COMMAND > command && srun -n1 --partition=gpu --gres=gpu:1 /bin/bash ./command

srun: job 3611 queued and waiting for resources
srun: job 3611 has been allocated resources
[1,0]<stdout>:Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
[1,0]<stdout>:11493376/11490434 [==============================] - 0s 0us/step]<stdout
[1,0]<stdout>:Epoch 1/8
[1,0]<stdout>:7/7 - 15s - loss: 1.5711 - accuracy: 0.5389
[1,0]<stdout>:Epoch time : 15.407773494720459
[1,0]<stdout>:Images/sec: 3894.14
[1,0]<stdout>:Epoch 2/8
[1,0]<stdout>:7/7 - 1s - loss: 0.6381 - accuracy: 0.7957
[1,0]<stdout>:Epoch time : 0.5352449417114258
[1,0]<stdout>:Images/sec: 112098.21
[1,0]<stdout>:Epoch 3/8
[1,0]<stdout>:7/7 - 1s - loss: 0.4345 - accuracy: 0.8677
[1,0]<stdout>:Epoch time : 0.5374562740325928
[1,0]<stdout>:Images/sec: 111636.99
[1,0]<stdout>:Epoch 4/8
[1,0]<stdout>:7/7 - 1s - loss: 0.3367 - accuracy: 0.8999
[1,0]<stdout>:Epoch time : 0.5405910015106201
[1,0]<stdout>:Images/sec: 110989.64
[1,0]<stdout>:Epoch 5/8
[1,0]<stdout>:7/7 - 1s - loss: 0.2748 - 

In [4]:
# 2 GPUs 
COMMAND = SINGULARITY_RUN + ' horovodrun -np 2 --mpi-args="--oversubscribe" python3 ../source_code/N1/cnn_fmnist.py --batch-size=8192 2> /dev/null' 
!echo  $COMMAND > command && srun --ntasks-per-node=2 -n1 --partition=gpu --gres=gpu:2 /bin/bash ./command

srun: job 3617 queued and waiting for resources
srun: job 3617 has been allocated resources
[1,1]<stdout>:Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
11493376/11490434 [==============================] - 0s 0us/step]<stdout
[1,0]<stdout>:Epoch 1/8
[1,0]<stdout>:3/3 - 22s - loss: 2.2004 - accuracy: 0.2774
[1,0]<stdout>:Epoch time : 21.74643063545227
[1,0]<stdout>:Images/sec: 2759.07
[1,0]<stdout>:Epoch 2/8
[1,0]<stdout>:3/3 - 1s - loss: 1.6737 - accuracy: 0.5839
[1,0]<stdout>:Epoch time : 1.0899956226348877
[1,0]<stdout>:Images/sec: 55046.09
[1,0]<stdout>:Epoch 3/8
[1,0]<stdout>:3/3 - 0s - loss: 1.0815 - accuracy: 0.6881
[1,0]<stdout>:Epoch time : 0.3508951663970947
[1,0]<stdout>:Images/sec: 170991.24
[1,0]<stdout>:Epoch 4/8
[1,0]<stdout>:3/3 - 0s - loss: 0.7905 - accuracy: 0.7485
[1,0]<stdout>:Epoch time : 0.35965800285339355
[1,0]<stdout>:Images/sec: 166825.15
[1,0]<stdout>:Epoch 5/8
[1,0]<stdout>:3/3 - 0s - loss: 0.5942 - accuracy: 0.814

In [6]:
# 4 GPUs
COMMAND = SINGULARITY_RUN + ' horovodrun -np 4 --mpi-args="--oversubscribe" python3 ../source_code/N1/cnn_fmnist.py --batch-size=8192 2> /dev/null' 
!echo  $COMMAND > command && srun --ntasks-per-node=4 -n1 --partition=gpu --gres=gpu:4 /bin/bash ./command

srun: job 3638 queued and waiting for resources
srun: job 3638 has been allocated resources
[1,0]<stdout>:Epoch 1/8
[1,0]<stdout>:1/1 - 27s - loss: 2.3059 - accuracy: 0.0927
[1,0]<stdout>:Epoch time : 26.99561834335327
[1,0]<stdout>:Images/sec: 2222.58
[1,0]<stdout>:Epoch 2/8
[1,0]<stdout>:1/1 - 2s - loss: 2.2376 - accuracy: 0.2363
[1,0]<stdout>:Epoch time : 2.40970516204834
[1,0]<stdout>:Images/sec: 24899.31
[1,0]<stdout>:Epoch 3/8
[1,0]<stdout>:1/1 - 1s - loss: 2.1101 - accuracy: 0.4230
[1,0]<stdout>:Epoch time : 1.4035429954528809
[1,0]<stdout>:Images/sec: 42748.96
[1,0]<stdout>:Epoch 4/8
[1,0]<stdout>:1/1 - 1s - loss: 1.9349 - accuracy: 0.4883
[1,0]<stdout>:Epoch time : 1.2914390563964844
[1,0]<stdout>:Images/sec: 46459.8
[1,0]<stdout>:Epoch 5/8
[1,0]<stdout>:1/1 - 1s - loss: 1.7329 - accuracy: 0.5594
[1,0]<stdout>:Epoch time : 1.1939504146575928
[1,0]<stdout>:Images/sec: 50253.34
[1,0]<stdout>:Epoch 6/8
[1,0]<stdout>:1/1 - 1s - loss: 1.5100 - accuracy: 0.6110
[1,0]<stdout>:Epoch t

Let us now aggregate the data into the below table below and calculate the scaling efficiency. Also ignore the first couple of iterations as they have graph building time and one-time operations in them.


The table below contains output of running the command on DGX-1 (A100) : 

|#GPUs |Samples/sec|Scaling efficiency|
|-|-|-|
|1| 109642| NA|
|2| 168561| ~77% |
|4| 262557| ~60%|



Now that we've run a demo and calculated the scaling efficiency of the model, we noticed a similar situation as in V100. Let us now get in-depth to understand our hardware environment and how it has the potential to affect the performance of our model.

***

## Licensing

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).

<div>
    <span style="float: left; width: 51%; text-align: right;">
        <a >1</a>
        <a href="2.1.System-Topology.ipynb">2</a>
        <a href="3.Hands-on-Multi-GPU.ipynb">3</a>
        <a href="4.Convergence.ipynb">4</a>
    </span>
    <span style="float: left; width: 49%; text-align: right;"><a href="2.1.System-Topology.ipynb">Next Notebook</a></span>
</div>

<br>
<p> <center> <a href="../Start_Here.ipynb">Home Page</a> </center> </p>


 



